# Installation and Import

In [36]:
# Transformers installation
! pip install transformers==4.28.0
! pip install datasets
! pip install sentencepiece
! pip install sacremoses
! pip install evaluate
! pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
# Transformers installation and import
import numpy as np
import pandas as pd
import evaluate
from transformers import pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
import torch.nn as nn
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import Dataset, load_metric
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification
warnings.filterwarnings('ignore')

# Part 1. Preprocessing a Custom Dataset

## Preprocess data with CSV


*   https://huggingface.co/learn/nlp-course/chapter5/3?fw=pt

*   https://huggingface.co/docs/datasets/v1.1.1/loading_datasets.html

*   https://colab.research.google.com/github/huggingface/notebooks/blob/master/course/en/chapter5/section3.ipynb#scrollTo=JRn46d_dxNHP







In [4]:
# To resolve CSV error in MS Excel, save files to 
# Preprocess data with Tab delimiter
from datasets import load_dataset
import csv
#train_df = pd.read_csv("COVID_Train_Set.csv", header=0, index_col=0, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, engine='python' )

#test_df = pd.read_csv("COVID_Test_Set.csv", header=0, encoding="utf-8", delimiter=",", quoting=csv.QUOTE_NONE, error_bad_lines=False, engine='python' )

#data_files = {"train": train_df, "test": test_df}

covid_dataset = load_dataset("csv", data_files={'train': 'COVID_Train_Set.csv', 'test': 'COVID_Test_Set.csv'}, delimiter=",")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-e10689e904786462/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
 #Shuffle and select the first 1000 samples
covid_sample = covid_dataset["train"].shuffle(seed=10).select(range(75))
covid_sample[:5]

{'sentence': ['\x0cmedRxiv preprint doi: https://doi.org/10.1101/2021.04.12.21255284; this version posted April 19, 2021. The copyright holder for this preprint (which was not certified by peer review) is the author/funder, who has granted medRxiv a license to display the preprint in perpetuity. This article is a US Government work. It is not subject to copyright under 17 USC 105 and is also made available for use under a CC0 license.',
  'ABSTRACTS',
  'PREDICTING AND TREND ANALYSIS OF COVID-19 CASES',
  'Supply chain management',
  'Phylogenetic Assignment of Named Global Outbreak LINeages (PANGOLIN, https://pangolin.cog-uk.io) to determine clade and lineage.'],
 'label': [0, 0, 1, 0, 1],
 'data': [None, None, 'COVID-19', None, 'Outbreak'],
 'regex': [False, False, True, False, True],
 'subset': ['Train', 'Train', 'Train', 'Train', 'Train']}

In [6]:
# Creates AttributeError: 'NoneType' object has no attribute 'lower'

def lowercase_sentence(example):
    return {"sentence": example["sentence"].lower()}


covid_dataset.map(lowercase_sentence)


Map:   0%|          | 0/50428 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:7                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/dataset_dict.py:851 in map                      │
│                                                                                                  │
│    848 │   │   if cache_file_names is None:                                                      │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│ ❱  851 │   │   │   {                                                                             │
│    852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/dataset_dict.py:852 in <dictcomp>               │
│                                                                                                  │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│    851 │   │   │   {                                                                             │
│ ❱  852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│    855 │   │   │   │   │   with_rank=with_rank,                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:578 in wrapper                 │
│                                                                                                  │
│    575 │   │   else:                                                                             │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    577 │   │   # apply actual function                                                           │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    580 │   │   for dataset in datasets:                                                          │
│    581 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:543 in wrapper                 │
│                                                                                                  │
│    540 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    541 │   │   }                                                                                 │
│    542 │   │   # apply actual function                                                           │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    544 │   │   datasets: List["Dataset"] = list(out.values(

In [7]:
def filter_nones(x):
    return x["sentence"] is not None

In [8]:
(lambda x: x * x)(3)

9

In [10]:
(lambda base, height: 0.5 * base * height)(4, 8)

16.0

In [11]:
covid_dataset = covid_dataset.filter(lambda x: x["sentence"] is not None)

Filter:   0%|          | 0/50428 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16255 [00:00<?, ? examples/s]

In [12]:
covid_dataset = covid_dataset.map(lowercase_sentence)
# Check that lowercasing worked
covid_dataset["train"]["sentence"][:3]

Map:   0%|          | 0/50299 [00:00<?, ? examples/s]

Map:   0%|          | 0/16200 [00:00<?, ? examples/s]

['review article',
 'yonsei med j 2022 nov;63(11):977-983 https://doi.org/10.3349/ymj.2022.0383',
 'pissn: 0513-5796 · eissn: 1976-2437']

In [13]:
def compute_sentence_length(example):
    return {"sentence_length": len(example["sentence"].split())}

In [14]:
covid_dataset = covid_dataset.map(compute_sentence_length)
# Inspect the first training example
covid_dataset["train"][0]

Map:   0%|          | 0/50299 [00:00<?, ? examples/s]

Map:   0%|          | 0/16200 [00:00<?, ? examples/s]

{'sentence': 'review article',
 'label': 0,
 'data': None,
 'regex': False,
 'subset': 'Train',
 'sentence_length': 2}

In [15]:
covid_dataset["train"].sort("sentence_length")[:3]

{'sentence': ['\x0c', '\x0c', '\x0c'],
 'label': [0, 0, 0],
 'data': [None, None, None],
 'regex': [False, False, False],
 'subset': ['Train', 'Train', 'Train'],
 'sentence_length': [0, 0, 0]}

In [16]:
covid_dataset = covid_dataset.filter(lambda x: x["sentence_length"] > 30)
print(covid_dataset.num_rows)

Filter:   0%|          | 0/50299 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16200 [00:00<?, ? examples/s]

{'train': 12744, 'test': 2426}


In [17]:
import html

text = "I&#039;m a transformer called BERT"
html.unescape(text)

"I'm a transformer called BERT"

In [18]:
covid_dataset = covid_dataset.map(lambda x: {"sentence": html.unescape(x["sentence"])})

Map:   0%|          | 0/12744 [00:00<?, ? examples/s]

Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

In [19]:
new_covid_dataset = covid_dataset.map(
    lambda x: {"sentence": [html.unescape(o) for o in x["sentence"]]}, batched=True
)

Map:   0%|          | 0/12744 [00:00<?, ? examples/s]

Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

In [20]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

In [21]:
%time tokenized_dataset = covid_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12744 [00:00<?, ? examples/s]

Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

CPU times: user 9.42 s, sys: 194 ms, total: 9.61 s
Wall time: 7.82 s


In [22]:
slow_tokenizer = AutoTokenizer.from_pretrained("bert-base-cased", use_fast=False)


def slow_tokenize_function(examples):
    return slow_tokenizer(examples["sentence"], truncation=True)


tokenized_dataset = covid_dataset.map(slow_tokenize_function, batched=True, num_proc=8)

Map (num_proc=8):   0%|          | 0/12744 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/2426 [00:00<?, ? examples/s]

In [23]:
def tokenize_and_split(examples):
    return tokenizer(
        examples["sentence"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )

In [24]:
result = tokenize_and_split(covid_dataset["train"][0])
[len(inp) for inp in result["input_ids"]]

[92]

In [25]:
# Causes error: ArrowInvalid: Column 6 named input_ids expected length 1000 but got length 1976
tokenized_dataset = covid_dataset.map(tokenize_and_split, batched=True)

Map:   0%|          | 0/12744 [00:00<?, ? examples/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/dataset_dict.py:851 in map                      │
│                                                                                                  │
│    848 │   │   if cache_file_names is None:                                                      │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│ ❱  851 │   │   │   {                                                                             │
│    852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/dataset_dict.py:852 in <dictcomp>               │
│                                                                                                  │
│    849 │   │   │   cache_file_names = {k: None for k in self}                                    │
│    850 │   │   return DatasetDict(                                                               │
│    851 │   │   │   {                                                                             │
│ ❱  852 │   │   │   │   k: dataset.map(                                                           │
│    853 │   │   │   │   │   function=function,                                                    │
│    854 │   │   │   │   │   with_indices=with_indices,                                            │
│    855 │   │   │   │   │   with_rank=with_rank,                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:578 in wrapper                 │
│                                                                                                  │
│    575 │   │   else:                                                                             │
│    576 │   │   │   self: "Dataset" = kwargs.pop("self")                                          │
│    577 │   │   # apply actual function                                                           │
│ ❱  578 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    579 │   │   datasets: List["Dataset"] = list(out.values()) if isinstance(out, dict) else [ou  │
│    580 │   │   for dataset in datasets:                                                          │
│    581 │   │   │   # Remove task templates if a column mapping of the template is no longer val  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:543 in wrapper                 │
│                                                                                                  │
│    540 │   │   │   "output_all_columns": self._output_all_columns,                               │
│    541 │   │   }                                                                                 │
│    542 │   │   # apply actual function                                                           │
│ ❱  543 │   │   out: Union["Dataset", "DatasetDict"] = func(self, *args, **kwargs)                │
│    544 │   │   datasets: List["Dataset"] = list(out.values(

In [26]:
tokenized_dataset = covid_dataset.map(
    tokenize_and_split, batched=True, remove_columns=covid_dataset["train"].column_names
)

Map:   0%|          | 0/12744 [00:00<?, ? examples/s]

Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

In [27]:
len(tokenized_dataset["train"]), len(covid_dataset["train"])

(27420, 12744)

In [28]:
def tokenize_and_split(examples):
    result = tokenizer(
        examples["sentence"],
        truncation=True,
        max_length=128,
        return_overflowing_tokens=True,
    )
    # Extract mapping between new and old indices
    sample_map = result.pop("overflow_to_sample_mapping")
    for key, values in examples.items():
        result[key] = [values[i] for i in sample_map]
    return result

In [29]:
tokenized_dataset = covid_dataset.map(tokenize_and_split, batched=True)
tokenized_dataset

Map:   0%|          | 0/12744 [00:00<?, ? examples/s]

Map:   0%|          | 0/2426 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'sentence_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27420
    })
    test: Dataset({
        features: ['sentence', 'label', 'data', 'regex', 'subset', 'sentence_length', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4073
    })
})

In [30]:
covid_dataset.set_format("pandas")

In [31]:
covid_dataset["train"][:3]

,sentence,label,data,regex,subset,sentence_length
0,"1department of medicine, university of califor...",1,Infectious,True,Train,43
1,as soon as the first case of the omicron varia...,1,case,True,Train,162
2,after the first case of severe acute respirato...,1,case,True,Train,44


In [32]:
train_df = covid_dataset["train"][:]

In [33]:
frequencies = (
    train_df["sentence"]
    .value_counts()
    .to_frame()
    .reset_index()
    .rename(columns={"index": "sentence", "sentence": "frequency"})
)
frequencies.head()

,sentence,frequency
0,"10991557, 2022, s2, downloaded from https://o...",625
1,"14698749, 2022, s4, downloaded from https://on...",49
2,"14401746, 2022, s1, downloaded from https://o...",35
3,medrxiv preprint doi: https://doi.org/10.1101...,27
4,medrxiv preprint doi: https://doi.org/10.1101...,27


In [34]:
from datasets import Dataset

freq_dataset = Dataset.from_pandas(frequencies)
freq_dataset

Dataset({
    features: ['sentence', 'frequency'],
    num_rows: 11758
})

In [ ]:
covid_dataset.reset_format()

In [ ]:
covid_dataset_clean = covid_dataset["train"].train_test_split(train_size=0.8, seed=42)
# Rename the default "test" split to "validation"
covid_dataset_clean["validation"] = covid_dataset_clean.pop("test")
# Add the "test" set to our `DatasetDict`
covid_dataset_clean["test"] = covid_dataset["test"]
covid_dataset_clean

In [ ]:
covid_dataset_clean.save_to_disk("covid-reviews")

In [ ]:
from datasets import load_from_disk

covid_dataset_reloaded = load_from_disk("covid-reviews")
covid_dataset_reloaded

In [ ]:
for split, dataset in covid_dataset_clean.items():
    dataset.to_json(f"covid-reviews-{split}.jsonl")

In [ ]:
!head -n 1 covid-reviews-train.jsonl

In [ ]:
data_files = {
    "train": "covid-reviews-train.jsonl",
    "validation": "covid-reviews-validation.jsonl",
    "test": "covid-reviews-test.jsonl",
}
covid_dataset_reloaded = load_dataset("json", data_files=data_files)

In [ ]:
# Setup the model
checkpoint = "bert-base-uncased"
# Use a CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(device)

In [ ]:
# Setup the data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
print(data_collator)

In [ ]:
# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    metric = load_metric("f1")
    return metric.compute(predictions=preds, references=labels)

In [ ]:
#training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "epoch",   # evaluation strategy to adopt during training
    learning_rate=2e-5,             # learning rate
    per_device_train_batch_size=3,  # batch size per device during training
    per_device_eval_batch_size=3,   # batch size for evaluation
    num_train_epochs=1,             # total number of training epochs
    weight_decay=0.01,              # strength of weight decay
    push_to_hub=False,
)
print(training_args)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,                            
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
# Compute and print the F1 score
metric = load_metric("f1")
predictions = trainer.predict(tokenized_dataset["test"])
preds = predictions.predictions.argmax(-1)
f1_score = metric.compute(predictions=preds, references=predictions.label_ids)
print(f1_score)


In [ ]:
# Compute and print the F1 score
metric = load_metric("accuracy")
predictions = trainer.predict(tokenized_dataset["test"])
preds = predictions.predictions.argmax(-1)
accuracy_score = metric.compute(predictions=preds, references=predictions.label_ids)
print(accuracy_score)

# Part 2. Full Training


*   https://huggingface.co/docs/transformers/training
*   https://huggingface.co/learn/nlp-course/chapter3/4?fw=pt





In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

##  Prepare for training

In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(["sentence"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")
tokenized_dataset["train"].column_names

In [ ]:
["attention_mask", "input_ids", "labels", "token_type_ids"]

In [ ]:

# This is a new one
batch["labels"] = torch.tensor([1] * len(sentences))
print(batch["labels"])

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
print(data_collator)

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_dataset["train"], shuffle=True, batch_size=8, collate_fn=data_collator,
)
eval_dataloader = DataLoader(
    tokenized_dataset["test"], batch_size=8, collate_fn=data_collator,
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

In [ ]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

## The training loop

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

## The evaluation loop

In [ ]:
import evaluate

metric = evaluate.load("glue", "mrpc")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

## Supercharge your training loop with Accelerate

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import Accelerator
from transformers import AdamW, AutoModelForSequenceClassification, get_scheduler

accelerator = Accelerator()

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
optimizer = AdamW(model.parameters(), lr=3e-5)

train_dl, eval_dl, model, optimizer = accelerator.prepare(
    train_dataloader, eval_dataloader, model, optimizer
)

num_epochs = 3
num_training_steps = num_epochs * len(train_dl)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dl:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

In [ ]:
from accelerate import notebook_launcher

notebook_launcher(training_function)

## Part 3. Previous Code

# Preprocessing Data and Tokenization

* https://huggingface.co/docs/transformers/preprocessing
* https://huggingface.co/transformers/v3.0.2/preprocessing.html
* https://huggingface.co/docs/datasets/nlp_load
* https://huggingface.co/docs/datasets/nlp_process
* https://huggingface.co/docs/datasets/load_hub




## Load data into Pandas dataframe


In [ ]:
# Load data into Pandas dataframe
df = pd.read_csv("526892.csv", encoding="cp1252", index_col=0)
print("DataFrame:", df)

In [ ]:
'''
from datasets import Dataset
from datasets import load_dataset
import random


def read_csv_file(file_path):
    return pd.read_csv(file_path, header=0, index_col=0, encoding="utf-8", delimiter="\t", quoting=csv.QUOTE_NONE, on_bad_lines="skip", engine='python')

train_df = read_csv_file("COVID_Train_Set.csv")
print(train_df)
test_df = read_csv_file("COVID_Test_Set.csv")
print(test_df)

data_files = {"train": train_df, "test": test_df}
print(data_files)

covid_dataset = Dataset.from_dict(data_files)
print(covid_dataset)

# Convert the train data list to a dataset object
covid_dataset["train"] = Dataset.from_dict(covid_dataset["train"])

# Shuffle and select the first 1000 samples
#covid_sample = covid_dataset["train"].shuffle(seed=42).select(range(1000))

# Convert the dataset object back to a list if necessary
#covid_sample = covid_dataset.to_pydict()["sentence"]
'''

In [ ]:

'''from datasets import load_dataset
from datasets import Dataset
import random

data_files = {"train": "COVID_Train_Set.csv", "test": "COVID_Test_Set.csv"}
covid_dataset = load_dataset("csv", data_files=data_files, delimiter="\t")

# Convert the train data list to a dataset object
covid_dataset["train"] = Dataset.from_dict(covid_dataset["train"])

# Shuffle and select the first 1000 samples
covid_sample = covid_dataset["train"].shuffle(seed=42).select(range(1000))

# Convert the dataset object back to a list if necessary
covid_sample = covid_sample.to_pydict()["text"]

# Peek at the first few examples
'''

In [ ]:
'''from datasets import Dataset
from datasets import load_dataset
import random

def read_csv_file(file_path):
    return pd.read_csv(file_path, header=0, index_col=0, encoding="utf-8", delimiter="\t", quoting=csv.QUOTE_NONE, on_bad_lines="skip", engine='python')

train_df = read_csv_file("COVID_Train_Set.csv")
test_df = read_csv_file("COVID_Test_Set.csv")

data_files = {"train": train_df, "test": test_df}

covid_dataset = Dataset.from_dict(data_files)

# Convert the train data list to a dataset object
covid_dataset[0]["train"] = Dataset.from_dict(covid_dataset[0]["train"])
'''

## Process sentences as sequences

In [ ]:
# Process sentences as sequences
sentences = df.iloc[0:2, 0]
sequences = sentences.tolist()
print(sequences)

## Tokenization


*   https://albertauyeung.github.io/2020/06/19/bert-tokenization.html/
*   https://huggingface.co/docs/transformers/model_doc/auto






In [ ]:
# Define tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
print(tokenizer)

# Define tokenize function
def tokenize_function(example):
    return tokenizer(example["sentence"], truncation=True)

# Same as before
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


## Model Architecture and CUDA


In [ ]:
# Use a CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

# Setup the model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint).to(device)
#model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

model.classifier = nn.Linear(model.classifier.in_features, 1).to(device)

# Freeze all parameters except the final layer
for param in model.parameters():
    param.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True

# Print the modified model architecture
print(model)

# Convert label list to tensor and one-hot encode it

In [ ]:
# Convert label list to tensor and one-hot encode it
labels = torch.tensor(list(df.iloc[:5, 1]))
print("Labels:", labels)
labels_one_hot = torch.nn.functional.one_hot(labels.long(), num_classes=2)
print("Labels One Hot", labels_one_hot)



## Convert data to Dataset

In [ ]:
# Convert data to Dataset format 1
train_data_dict = {"sentence": list(df.iloc[:5, 0]), "label": list(df.iloc[:5, 1])}
print("Train Data Dict:", train_data_dict)
dev_data_dict = {"sentence": list(df.iloc[5:7, 0]), "label": list(df.iloc[5:7, 1])}
print("Dev Data Dict:", train_data_dict)
test_data_dict = {"sentence": list(df.iloc[7:10, 0]), "label": list(df.iloc[7:10, 1])}
print("Test Data Dict:", train_data_dict)
train_dataset = Dataset.from_dict(train_data_dict)
print("Train Dataset:", train_dataset)
dev_dataset = Dataset.from_dict(dev_data_dict)
print(dev_dataset)
test_dataset = Dataset.from_dict(test_data_dict)
print(test_dataset)

# Convert data to Dataset format 2
data_dict = {"sentence": list(df.iloc[0:2, 0]), "label": list(df.iloc[0:2, 1])}
raw_dataset = Dataset.from_dict(data_dict)

# Create dictionary with "sentence1" and "label" as keys and the respective columns from the pandas dataframe as values
data_dict = {"sentence": list(df.iloc[0:10, 0]), "label": list(df.iloc[0:10, 1])}

# Convert dictionary to Dataset object 2
dataset = Dataset.from_dict(data_dict)
print (dataset)

## Tokenize sentences

In [ ]:
# Tokenize sentences
batch = tokenizer(list(df.iloc[0:10, 0]), padding=True, truncation=True, return_tensors="pt").to(device)
print(batch)

# Tokenize sentences
train_tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
print(train_tokenized_dataset)
dev_tokenized_dataset = dev_dataset.map(tokenize_function, batched=True)
print(dev_tokenized_dataset)
test_tokenized_dataset = test_dataset.map(tokenize_function, batched=True)
print(test_tokenized_dataset)


## Add labels to batch

In [ ]:
# Add labels to batch
batch["labels"] = torch.tensor(df.iloc[0:10, 1].values).unsqueeze(1).to(device)  # Add unsqueeze here
#batch["labels"] = torch.tensor(df.iloc[0:10, 1].values).to(device)  # Add unsqueeze here

# This is a new one
batch["labels"] = torch.tensor([1] * len(sequences))
print(batch["labels"])

## Model Output

In [ ]:
# Set output format
columns_to_return = ["input_ids", "attention_mask", "label"]
train_tokenized_dataset.set_format(type='torch', columns=columns_to_return)
print(train_tokenized_dataset)
dev_tokenized_dataset.set_format(type='torch', columns=columns_to_return)
print(dev_tokenized_dataset)
test_tokenized_dataset.set_format(type='torch', columns=columns_to_return)
print(test_tokenized_dataset)

## Data Collator


In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)
print(data_collator)

## DataLoader

In [ ]:
batch_size = 8
train_dataloader = torch.utils.data.DataLoader(train_tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)
dev_dataloader = torch.utils.data.DataLoader(dev_tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)
test_dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=batch_size, collate_fn=data_collator)

## Prepare data batch

In [ ]:
batch_size = 1
dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=batch_size)
print(dataloader)
for batch in dataloader:
    input_ids = batch["input_ids"]
    print(input_ids)
    attention_mask = batch["attention_mask"]
    print(attention_mask)
    label = batch["label"]
    print(label)
    inputs = {"input_ids": input_ids, "attention_mask": attention_mask, "labels": label.to(torch.long), "targets": targets}
    print(inputs)
    outputs = model(**inputs)
    print(outputs)
    loss = outputs.loss
    print(loss)
    logits = outputs.logits
    print(logits)

## Model Output Optimization and Calculation


In [ ]:
optimizer = AdamW(model.parameters())
#output = model(**batch).logits.squeeze(1)  # Squeeze the output to get [batch_size]
output = model(**batch).logits  # Squeeze the output to get [batch_size]
loss = nn.BCEWithLogitsLoss()(output, batch["labels"].float())
loss.backward()
optimizer.step()

# Finetuning the Model


*   https://huggingface.co/learn/nlp-course/chapter7/3?fw=tf
*   https://huggingface.co/learn/nlp-course/chapter3/3?fw=pt
*   https://huggingface.co/docs/transformers/v4.18.0/en/training



## Compute Metrics Function

In [ ]:
# Define compute_metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    metric = load_metric("f1")
    return metric.compute(predictions=preds, references=labels)

## Training Arguments

In [ ]:
#training_args = TrainingArguments("test-trainer", evaluation_strategy="epoch")
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "epoch",   # evaluation strategy to adopt during training
    learning_rate=2e-5,             # learning rate
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    num_train_epochs=3,             # total number of training epochs
    weight_decay=0.01,              # strength of weight decay
    push_to_hub=False,
)
print(training_args)

## Setup the trainer

In [ ]:
trainer = Trainer(
    model=model,                     
    args=training_args,              
    train_dataset=train_tokenized_dataset,         
    eval_dataset=dev_tokenized_dataset,            
    data_collator=data_collator,                     
    tokenizer=tokenizer,                            
    compute_metrics=compute_metrics,                  
)
print(trainer)

## Fine tune the model

In [ ]:
# Fine-tune the model
trainer.train()

## Evaluate on the test set


In [ ]:
# Evaluate the model on the test set
#predictions = trainer.predict(test_dataset)
predictions = trainer.predict(tokenized_datasets["test"])
#print(predictions.predictions.shape, predictions.label_ids.shape)
preds = np.argmax(predictions.predictions, axis=-1)
test_results = trainer.evaluate(test_tokenized_dataset)
print(test_results)

## Compute and print the F1 score

In [ ]:
# Compute and print the F1 score
metric = load_metric("f1")
predictions = trainer.predict(test_tokenized_dataset)
preds = predictions.predictions.argmax(-1)
f1_score = metric.compute(predictions=preds, references=predictions.label_ids)
print(f1_score)
